In [2]:
from collections import deque
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from python_package.data_feed import clean_labels, apply_dark_mode


%matplotlib qt5
plt.ion()
apply_dark_mode()

## Select file name
file_name = 'eeg.csv'
data = pd.read_csv('data/' + file_name)

## Update columns name
data_labels = ['Time [min]', 'Pressure [PSIg]','Flow Rate [m3/h]']
data_list_columns = clean_labels(data_labels)
data.columns = data_list_columns
data;


def build_frames(data: pd.DataFrame, label: str) -> pd.DataFrame:
    """Storage all the results of processing the deque object"""
    drop_elements = list(data[label].values)[-2::-1]
    dy = deque(data[label].values, maxlen=len(data))
    lista_frames = [dy.copy()]

    while len(drop_elements) > 0:
        dy.append(drop_elements.pop())
        lista_frames.append(dy.copy())

    row_keys = [f'{i}' for i in data.index]

    df = pd.DataFrame(lista_frames, columns=row_keys)

    return df


## Select n: whole_frame / n = window
n = 1
window = int(len(data)/n)

## Choose label for x
label = data_list_columns[0]
df_x = build_frames(data, label).iloc[:,:window]

## Choose label for y1
label = data_list_columns[1]
df_pressure = build_frames(data, label).iloc[:,:window]

## Choose label for y2
label = data_list_columns[2]
df_flow_rate = build_frames(data, label).iloc[:,:window]

fig, (ax1, ax2) = plt.subplots(nrows=2, sharex=True)

def plot_data(df, ax, speed=100):
    # fig, ax = plt.subplots()
    line, = ax.plot([], [])
    
    def update(i):
        ax.clear()
        line, = ax.plot(df.iloc[i])
        return line, 

    return FuncAnimation(fig, update, frames=len(df), interval=speed, blit=True)


plt.tick_params(
    axis='x', 
    which='both', 
    bottom=True,
    top=False,
    labelbottom=False,
    )

ani1 = plot_data(df_pressure, ax1, speed=10)
ani2 = plot_data(df_flow_rate, ax2, speed=10)